# Introduction

In this notebook, our focus is three-fold: firstly, to demonstrate how one can seamlessly connect to OpenAI's GPT-3.5 using our existing connector, secondly, to showcase how to effectively create Moonshot's recipe and cookbook, and lastly to run benchmarks leveraging the Moonshot library.

* Create an endpoint
* Create a recipe
* Create a cookbook
* List and run a recipe
* List and run a cookbook

## Pre-requisite

If you have not create a virtual environment with this notebook, we suggest creating one to avoid any conflicts in the Python libraries. Once you have created the virtual environment, install all the requirements using the following command:

```pip install -r requirements.txt```

## Import and Environment Variables

Import Moonshot library to use in Jupyter notebook

In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import sys, os, json
sys.path.insert(0, '../')

import asyncio
from moonshot.api import (
    api_create_recipe,
    api_create_cookbook,
    api_create_endpoint,
    api_create_recipe_executor,
    api_create_cookbook_executor,
    api_create_session,
    api_get_session,
    api_get_all_connector_type,
    api_get_all_endpoint,
    api_get_all_cookbook,
    api_get_all_recipe,
    api_get_all_executor,
    api_get_all_session_detail,
    api_get_all_prompt_template_detail,
    api_get_all_context_strategy_name,
    api_get_session_chats_by_session_id,
    api_load_executor,
    api_set_environment_variables,
    api_send_prompt,
    api_update_context_strategy,
    api_update_prompt_template,
)


### To prettify the tables, we use Python library - rich ###
from rich.columns import Columns
from rich.console import Console
from rich.panel import Panel
from rich.table import Table

moonshot_path = "../moonshot/data/"

env = {
    "CONNECTORS_ENDPOINTS": os.path.join(moonshot_path, "connectors-endpoints"),
    "CONNECTORS": os.path.join(moonshot_path, "connectors"),
    "RECIPES": os.path.join(moonshot_path, "recipes"),
    "COOKBOOKS": os.path.join(moonshot_path, "cookbooks"),
    "DATASETS": os.path.join(moonshot_path, "datasets"),
    "PROMPT_TEMPLATES": os.path.join(moonshot_path, "prompt-templates"),
    "METRICS": os.path.join(moonshot_path, "metrics"),
    "METRICS_CONFIG": os.path.join(moonshot_path, "metrics/metrics_config.json"),
    "CONTEXT_STRATEGY": os.path.join(moonshot_path, "context-strategy"),
    "RESULTS": os.path.join(moonshot_path, "results"),
    "DATABASES": os.path.join(moonshot_path, "databases"),
    "SESSIONS": os.path.join(moonshot_path, "sessions"),
}

api_set_environment_variables(env)

# initialise the global console
console = Console()

## Prettify Functions

These functions help to beautify the results from Moonshot libraries.

<a id='prettified_functions'></a>

In [2]:
def list_connector_types(connector_types):
    if connector_types:
        table = Table("No.", "Connector Type")
        for connector_id, connector_type in enumerate(connector_types, 1):
            table.add_section()
            table.add_row(str(connector_id), connector_type)
        console.print(table)
    else:
        console.print("[red]There are no connector types found.[/red]")
        
def list_endpoints(endpoints_list):
    if endpoints_list:
        table = Table(
            "No.",
            "Id",
            "Name",
            "Connector Type",
            "Uri",
            "Token",
            "Max calls per second",
            "Max concurrency",
            "Params",
            "Created Date",
        )
        for endpoint_id, endpoint in enumerate(endpoints_list, 1):
            (
                id,
                name,
                connector_type,
                uri,
                token,
                max_calls_per_second,
                max_concurrency,
                params,
                created_date,
            ) = endpoint.values()
            table.add_section()
            table.add_row(
                str(endpoint_id),
                id,
                name,
                connector_type,
                uri,
                token,
                str(max_calls_per_second),
                str(max_concurrency),
                str(params),
                created_date,
            )
        console.print(table)
    else:
        console.print("[red]There are no endpoints found.[/red]")

def list_recipes(recipes_list):
    if recipes_list:
        table = Table("No.", "Recipe", "Contains")
        for recipe_id, recipe in enumerate(recipes_list, 1):
            (
                id,
                name,
                description,
                tags,
                datasets,
                prompt_templates,
                metrics
            ) = recipe.values()
            recipe_info = f"[red]id: {id}[/red]\n\n[blue]{name}[/blue]\n{description}\n\nTags:\n{tags}"
            dataset_info = "[blue]Datasets[/blue]:" + "".join(
                f"\n{i + 1}. {item}" for i, item in enumerate(datasets)
            )
            prompt_templates_info = "[blue]Prompt Templates[/blue]:" + "".join(
                f"\n{i + 1}. {item}" for i, item in enumerate(prompt_templates)
            )
            metrics_info = "[blue]Metrics[/blue]:" + "".join(
                f"\n{i + 1}. {item}" for i, item in enumerate(metrics)
            )
            contains_info = (
                f"{dataset_info}\n{prompt_templates_info}\n{metrics_info}"
            )
            table.add_section()
            table.add_row(str(recipe_id), recipe_info, contains_info)
        console.print(table)
    else:
        console.print("[red]There are no recipes found.[/red]")

def list_cookbooks(cookbooks_list):
    if cookbooks_list:
        table = Table("No.", "Cookbook", "Recipes")
        for cookbook_id, cookbook in enumerate(cookbooks_list, 1):
            id, name, description, recipes = cookbook.values()
            cookbook_info = (
                f"[red]id: {id}[/red]\n\n[blue]{name}[/blue]\n{description}"
            )
            recipes_info = "\n".join(
                f"{i + 1}. {item}" for i, item in enumerate(recipes)
            )
            table.add_section()
            table.add_row(str(cookbook_id), cookbook_info, recipes_info)
        console.print(table)
    else:
        console.print("[red]There are no cookbooks found.[/red]")

def show_recipe_results(recipes, endpoints, recipe_results, results_file, duration):
    if recipe_results:
        # Display recipe results
        generate_recipe_table(recipes, endpoints, recipe_results)
        console.print(
            f"[blue]Results saved in {results_file}[/blue]"
        )
    else:
        console.print("[red]There are no results.[/red]")

    # Print run stats
    console.print(f"{'='*50}\n[blue]Time taken to run: {duration}s[/blue]\n{'='*50}")


def show_cookbook_results(cookbooks, endpoints, cookbook_results, results_file, duration):
    if cookbook_results:
        # Display recipe results
        generate_cookbook_table(cookbooks, endpoints, cookbook_results)
        console.print(
            f"[blue]Results saved in {results_file}[/blue]"
        )
    else:
        console.print("[red]There are no results.[/red]")
    
    # Print run stats
    console.print(f"{'='*50}\n[blue]Time taken to run: {duration}s[/blue]\n{'='*50}")


def generate_recipe_table(
        recipes: list, endpoints: list, results: dict
    ) -> None:
    table = Table("", "Recipe", *endpoints)
    for recipe_index, recipe in enumerate(recipes, 1):
        endpoint_results = list()
        for endpoint in endpoints:
            tmp_results = {}
            for result_key, result_value in results[recipe].items():
                if set((endpoint, recipe)).issubset(result_key):
                    result_ep, result_recipe, result_ds, result_pt = result_key
                    tmp_results[(result_ds, result_pt)] = result_value['results']
            endpoint_results.append(str(tmp_results))
        table.add_section()
        table.add_row(str(recipe_index), recipe, *endpoint_results)
    # Display table
    console.print(table)

def generate_cookbook_table(cookbooks, endpoints: list, results: dict) -> None:
    table = Table("", "Cookbook", "Recipe", *endpoints)
    index = 1
    for cookbook in cookbooks:
        cookbook_results = results[cookbook]
        for recipe_name, recipe_results in cookbook_results.items():
            endpoint_results = list()
            for endpoint in endpoints:
                tmp_results = {}
                for result_key, result_value in results[cookbook][recipe_name].items():
                    if set((endpoint, recipe_name)).issubset(result_key):
                        result_ep, result_recipe, result_ds, result_pt = result_key
                        tmp_results[(result_ds, result_pt)] = result_value["results"]
                endpoint_results.append(str(tmp_results))
            table.add_section()
            table.add_row(str(index), cookbook, recipe_name, *endpoint_results)
            index += 1
    # Display table
    console.print(table)

def list_runs(runs_list):
    if runs_list:
        table = Table("No.", "Run id", "Contains")
        for run_index, run_data in enumerate(runs_list, 1):
            (
                run_id,
                run_type,
                start_time,
                end_time,
                duration,
                db_file,
                error_messages,
                results_file,
                recipes,
                cookbooks,
                endpoints,
                num_of_prompts,
                results,
                status,
                progress_callback_func,
            ) = run_data.values()
            run_info = f"[red]id: {run_id}[/red]\n"
    
            contains_info = ""
            if recipes:
                contains_info += f"[blue]Recipes:[/blue]\n{recipes}\n\n"
            elif cookbooks:
                contains_info += f"[blue]Cookbooks:[/blue]\n{cookbooks}\n\n"
            contains_info += f"[blue]Endpoints:[/blue]\n{endpoints}\n\n"
            contains_info += (
                f"[blue]Number of Prompts:[/blue]\n{num_of_prompts}\n\n"
            )
            contains_info += f"[blue]Database path:[/blue]\n{db_file}"
    
            table.add_section()
            table.add_row(str(run_index), run_info, contains_info)
        console.print(table)
    else:
        console.print("[red]There are no runs found.[/red]")

def list_sessions(session_list):
    if session_list:
        table = Table("No.", "Session id", "Contains")
        for session_index, session_data in enumerate(session_list, 1):
            (
                session_id,
                name,
                description,
                endpoints,
                created_epoch,
                created_datetime,
                context_strategy,
                prompt_template,
                chat_ids,
            ) = session_data.values()
            session_info = f"[red]id: {session_id}[/red]\n"
    
            contains_info = ""
            contains_info += f"[blue]Name:[/blue]\n{name}\n\n"
            contains_info += f"[blue]Description:[/blue]\n{description}\n\n"
            contains_info += f"[blue]Created:[/blue]\n{created_datetime}\n\n"

            if context_strategy:
                contains_info += f"[blue]Context Strategy:[/blue]\n{context_strategy}\n\n"
            else:
                contains_info += f"[blue]Context Strategy:[/blue]\nNone\n\n"

            if prompt_template:
                contains_info += f"[blue]Prompt Template:[/blue]\n{prompt_template}\n\n"
            else:
                contains_info += f"[blue]Prompt Template:[/blue]\nNone\n\n"
                
            contains_info += f"[blue]Chat IDs:[/blue]\n{chat_ids}\n\n"
    
            table.add_section()
            table.add_row(str(session_index), session_info, contains_info)
        console.print(table)
    else:
        console.print("[red]There are no sessions found.[/red]")

def list_context_strategy(context_strategies):
    if context_strategies:
        table = Table("No.", "Context Strategies")
        for ct_index, ct_data in enumerate(context_strategies, 1):
            table.add_section()
            table.add_row(str(ct_index), ct_data)
        console.print(table)
    else:
        console.print("[red]There are no context strategies found.[/red]")

def list_prompt_templates(prompt_templates):
    if prompt_templates:
        table = Table("No.", "Prompt Templates")
        for pt_index, pt_data in enumerate(prompt_templates, 1):
            (
                name,
                description,
                template,
            ) = pt_data.values()
    
            contains_info = ""
            contains_info += f"[blue]Name:[/blue]\n{name}\n\n"
            contains_info += f"[blue]Description:[/blue]\n{description}\n\n"
            contains_info += f"[blue]Template:[/blue]\n{template}\n\n"
            table.add_section()
            table.add_row(str(pt_index), contains_info)
        console.print(table)
    else:
        console.print("[red]There are no prompt templates found.[/red]")

def show_session_chats(session_chats):
    if session_chats:
        table = Table("No.", "Endpoint", "Contains")
        for chat_index, chat_data in enumerate(session_chats, 1):
            (
                chat_id,
                endpoint,
                chat_history
            ) = chat_data.values()
            for chat_history_index, chat_history_data in enumerate(chat_history, 1):
                (
                    chat_record_id,
                    conn_id,
                    context_strategy,
                    prompt_template,
                    prompt,
                    prepared_prompt,
                    predicted_result,
                    duration,
                    prompt_time
                ) = chat_history_data.values()
                
                contains_info = ""
                contains_info += f"[blue]Chat Record Id:[/blue]\n{chat_record_id}\n\n"
                if conn_id:
                    contains_info += f"[blue]Connection Id:[/blue]\n{conn_id}\n\n"
                else:
                    contains_info += f"[blue]Connection Id:[/blue]\nNone\n\n"

                if context_strategy:
                    contains_info += f"[blue]Context Strategy:[/blue]\n{context_strategy}\n\n"
                else:
                    contains_info += f"[blue]Context Strategy:[/blue]\nNone\n\n"
                
                if prompt_template:
                    contains_info += f"[blue]Prompt Template:[/blue]\n{prompt_template}\n\n"
                else:
                    contains_info += f"[blue]Prompt Template:[/blue]\nNone\n\n"
                    
                contains_info += f"[blue]Prompt[/blue]\n{prompt}\n\n"
                contains_info += f"[blue]Prepared Prompt:[/blue]\n{prepared_prompt}\n\n"
                contains_info += f"[blue]Predicted Result:[/blue]\n{predicted_result}\n\n"
                contains_info += f"[blue]Duration:[/blue]\n{duration}s\n\n"
                contains_info += f"[blue]Prompt Time:[/blue]\n{prompt_time}\n\n"
                table.add_section()
                table.add_row(str(chat_index), endpoint, contains_info)
        console.print(table)
    else:
        console.print("[red]There are no session chats found.[/red]")

def show_session(session_instance):
    if session_instance:
        metadata = session_instance.metadata
        table = Table("Session Id", "Session Info")
        contains_info = ""
        contains_info += f"[blue]Name:[/blue]\n{metadata.name}\n\n"
        contains_info += f"[blue]Description:[/blue]\n{metadata.description}\n\n"
        contains_info += f"[blue]Endpoints:[/blue]\n{metadata.endpoints}\n\n"
        if metadata.context_strategy:
            contains_info += f"[blue]Context Strategy:[/blue]\n{metadata.context_strategy}\n\n"
        else:
            contains_info += f"[blue]Context Strategy:[/blue]\nNone\n\n"
        
        if metadata.prompt_template:
            contains_info += f"[blue]Prompt Template:[/blue]\n{metadata.prompt_template}\n\n"
        else:
            contains_info += f"[blue]Prompt Template:[/blue]\nNone\n\n"

        table.add_section()
        table.add_row(metadata.session_id, contains_info)
        console.print(table)
    else:
        console.print("[red]Session is not found[/red]")


## Create an endpoint

An endpoint in the context of Moonshot refers to the actual configuration used to connect to a model (i.e. connector). Before an endpoint can be created, the `connector` must exist in the list of the connector.

In this section, you will learn how to create an endpoint using an existing connector that we have included in Moonshot.

### Connector Type

We can list the connectors available in Moonshot using `api_get_all_connector_type()` as shown in the cell below. A connector details the following two mandatory behaviors:

1. How to call the model? (For developers, checkout the function `get_response()` in one of the connector python files in `moonshot\data\connectors\`)
   
2. How to process the response return by the model? (For developers, checkout the function `_process_response()`)

In [3]:
connection_types = api_get_all_connector_type()
connection_types

['hf-llama2-13b-gptq',
 'openai-gpt4',
 'claude2',
 'openai-gpt35',
 'openai-gpt35-turbo-16k',
 'hf-gpt2']

#### Beautify the results

The results from Moonshot library can be prettified using `rich` library. We have provided these prettified functions in this [cell](#prettified_functions).

In [4]:
list_connector_types(connection_types)

┏━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ No. ┃ Connector Type         ┃
┡━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 1   │ hf-llama2-13b-gptq     │
├─────┼────────────────────────┤
│ 2   │ openai-gpt4            │
├─────┼────────────────────────┤
│ 3   │ claude2                │
├─────┼────────────────────────┤
│ 4   │ openai-gpt35           │
├─────┼────────────────────────┤
│ 5   │ openai-gpt35-turbo-16k │
├─────┼────────────────────────┤
│ 6   │ hf-gpt2                │
└─────┴────────────────────────┘

### Endpoint

In this notebook, we will evaluate `openai-gpt35`. To connect to a model, we need to create an endpoint to the model.

To create a new endpoint, we can use `api_create_endpoint()`.

Once an endpoint has been added to Moonshot, we can use this endpoint to evaluate the model later when we run our benchmark.

In [5]:
endpoints_list = api_get_all_endpoint()
list_endpoints(endpoints_list)

┏━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━┓
┃     ┃            ┃            ┃ Connector  ┃     ┃            ┃ Max calls  ┃ Max       ┃            ┃ Created   ┃
┃ No. ┃ Id         ┃ Name       ┃ Type       ┃ Uri ┃ Token      ┃ per second ┃ concurre… ┃ Params     ┃ Date      ┃
┡━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━┩
│ 1   │ my-openai… │ my gpt35   │ openai-gp… │     │            │ 100        │ 100       │ {'timeout… │ 2024-03-… │
│     │            │            │            │     │            │            │           │ 300,       │ 15:28:13  │
│     │            │            │            │     │            │            │           │ 'allow_re… │           │
│     │            │            │            │     │            │            │           │ True,      │           │
│     │            │            │            │     │            │            │           │ 'num_of_r… │           │
│     │            │            │            │     │            │            │           │ 2}         │           │
├─────┼────────────┼────────────┼────────────┼─────┼────────────┼────────────┼───────────┼────────────┼───────────┤
│ 2   │ test-open… │ test-open… │ openai-gp… │     │ sk-D8AECa… │ 10         │ 2         │ {'tempera… │ 2024-03-… │
│     │            │            │            │     │            │            │           │ 0}         │ 17:24:34  │
└─────┴────────────┴────────────┴────────────┴─────┴────────────┴────────────┴───────────┴────────────┴───────────┘

In [6]:
api_create_endpoint(
    "test-openai-endpoint", # name: give it a name to retrieve it later
    "openai-gpt35", # connector_type: the model that we want to evaluate
    "", # uri: not required as we use OpenAI library to connect to their models.
    "ADD_NEW_TOKEN_HERE", # token: access token
    10, # max_calls_per_second: the number of max calls per second
    2, # max_concurrency: the number of concurrent call at any one time,
    {
        "temperature": 0
    } # params: any additional required for this model
)

# Refresh
endpoints_list = api_get_all_endpoint()
list_endpoints(endpoints_list)

┏━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━┓
┃     ┃            ┃            ┃ Connector  ┃     ┃            ┃ Max calls  ┃ Max       ┃            ┃ Created   ┃
┃ No. ┃ Id         ┃ Name       ┃ Type       ┃ Uri ┃ Token      ┃ per second ┃ concurre… ┃ Params     ┃ Date      ┃
┡━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━┩
│ 1   │ my-openai… │ my gpt35   │ openai-gp… │     │            │ 100        │ 100       │ {'timeout… │ 2024-03-… │
│     │            │            │            │     │            │            │           │ 300,       │ 15:28:13  │
│     │            │            │            │     │            │            │           │ 'allow_re… │           │
│     │            │            │            │     │            │            │           │ True,      │           │
│     │            │            │            │     │            │            │           │ 'num_of_r… │           │
│     │            │            │            │     │            │            │           │ 2}         │           │
├─────┼────────────┼────────────┼────────────┼─────┼────────────┼────────────┼───────────┼────────────┼───────────┤
│ 2   │ test-open… │ test-open… │ openai-gp… │     │ ADD_NEW_T… │ 10         │ 2         │ {'tempera… │ 2024-03-… │
│     │            │            │            │     │            │            │           │ 0}         │ 22:04:31  │
└─────┴────────────┴────────────┴────────────┴─────┴────────────┴────────────┴───────────┴────────────┴───────────┘

# Create a recipe

A recipe contains all the ingredients required to run a benchmark. It gives Moonshot step-by-step instructions on what to do with those ingredients to run a successful benchmark on the selected model.

The recipe includes the following important details:

1. Name of the recipe (to be used later)
2. Dataset
3. Metric(s)
4. Prompt template (s) (if any)

In this notebook, we will create a test dataset to add to our new recipe. All datasets can be found in `moonshot\data\datasets`. 

In [7]:
test_dataset = {
    "name": "test-dataset",
    "description": "This dataset contains questions on general items and its category.",
    "keywords": [
        "general"
    ],
    "categories": [
        "capability"
    ],
    "examples": [
        {
            "input": "What is an apple?",
            "target": "Fruit"
        },
        {
            "input": "What is a chair?",
            "target": "Furniture"
        },
        {
            "input": "What is a laptop?",
            "target": "Electronic"
        },
        {
            "input": "What is a biscuit?",
            "target": "Food"
        }
        ,
        {
            "input": "What is a pear?",
            "target": "Fruit"
        }
    ]
}

# to change later when notebook is shifted
in_file = "../moonshot/data/datasets/test-dataset.json"
json.dump(test_dataset, open(in_file, "w+"), indent=2)

In this notebook, we create a new prompt template to use with this dataset. When this prompt template is activated, an example prompt will be sent to the model in this form using the dataset above:

```
Answer this question:
What is an apple?
A:
```

In [8]:
prompt_template = {
    "name": "Simple Question Answering Template",
    "description": "This is a simple question and answering template.",
    "template": "Answer this question:\n{{ prompt }}\nA:"
}

in_file = "../moonshot/data/prompt-templates/test-prompt-template.json"
json.dump(prompt_template, open(in_file, "w+"), indent=2)

To add a new recipe, we can use `api_create_recipe`. We will use our dataset and prompt template from the previous two cells in this recipe. 

In [9]:
api_create_recipe(
    "Item Category",
    "This recipe is created to test model's ability in answering question.",
    ["tag1"],
    ["test-dataset"],
    ["test-prompt-template"],
    ["exactstrmatch", 'rougescore']
)

recipes_list = api_get_all_recipe()
list_recipes(recipes_list)

┏━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ No. ┃ Recipe                                                 ┃ Contains                                         ┃
┡━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 1   │ id: squad-shifts-tnf                                   │ Datasets:                                        │
│     │                                                        │ 1. squad-shifts-tnf                              │
│     │ squad-shifts-tnf                                       │ Prompt Templates:                                │
│     │ Zero-shot reading comprehension on paragraphs and      │ Metrics:                                         │
│     │ questions from squadshifts. Augmented to true/false    │ 1. relaxstrmatch                                 │
│     │ statement.                                             │                                                  │
│     │                                                        │                                                  │
│     │ Tags:                                                  │                                                  │
│     │ []                                                     │                                                  │
├─────┼────────────────────────────────────────────────────────┼──────────────────────────────────────────────────┤
│ 2   │ id: tamil-kural-classification                         │ Datasets:                                        │
│     │                                                        │ 1. tamil-kural-classification                    │
│     │ TAMIL-KURAL-CLASSIFICATION                             │ Prompt Templates:                                │
│     │ This recipe is used to test the comprehension          │ 1. tamil-templatekuralclassification             │
│     │ abilities for the Thirukkural. Thirukkural is a        │ Metrics:                                         │
│     │ classic Tamil literature composed by the ancient Tamil │ 1. exactstrmatch                                 │
│     │ poet Thiruvalluvar. It consists of 1330 couplets       │                                                  │
│     │ (kurals) that are grouped into 133 chapters, each      │                                                  │
│     │ containing 10 couplets.                                │                                                  │
│     │                                                        │                                                  │
│     │ Tags:                                                  │                                                  │
│     │ ['tamil', 'text classification']                       │                                                  │
├─────┼────────────────────────────────────────────────────────┼──────────────────────────────────────────────────┤
│ 3   │ id: advglue-sst2                                       │ Datasets:                                        │
│     │                                                        │ 1. advglue-sst2-adv                              │
│     │ AdvGLUE-SST2                                           │ 2. advglue-sst2-benign                           │
│     │ Adversarially Perturbed. SST-2 consists of sentences   │ Prompt Templates:                                │
│     │ from movie reviews and human annotations of their      │ 1. advglue-templatesst2                          │
│     │ sentiment.                                             │ Metrics:                                         │
│     │                                                        │ 1. exactstrmatch                                 │
│     │ Tags:                                                  │                                                  │
│     │ ['robustness']                                  

# Create a cookbook

A cookbook can contain more than one recipes. It is meant to organise and group the recipes together so that a set of recipes can be used to evaluate a model. To add a cookbook, we use `api_create_cookbook`

In [10]:
api_create_cookbook(
    "test-category-cookbook",
    "This cookbook tests if the model is able to group items into different categories",
    ["item-category"]
)

cookbooks_list = api_get_all_cookbook()
list_cookbooks(cookbooks_list)

┏━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ No. ┃ Cookbook                                                   ┃ Recipes                                      ┃
┡━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 1   │ id: leaderboard-cookbook                                   │ 1. mmlu                                      │
│     │                                                            │ 2. truthfulqa-mcq                            │
│     │ Leaderboard Cookbook                                       │ 3. winogrande                                │
│     │ This cookbook, drawing inspiration from leaderboards like  │ 4. hellaswag                                 │
│     │ HF's OpenLLM and HELM, features popular benchmarks for     │ 5. arc                                       │
│     │ testing model capabilities, with results that may vary     │ 6. gsm8k                                     │
│     │ from actual leaderboard standings.                         │                                              │
├─────┼────────────────────────────────────────────────────────────┼──────────────────────────────────────────────┤
│ 2   │ id: cbbq-amb-cookbook                                      │ 1. cbbq-lite                                 │
│     │                                                            │                                              │
│     │ CBBQ (Ambiguous)                                           │                                              │
│     │ This is a cookbook that consists all the ambiguous         │                                              │
│     │ questions from CBBQ.                                       │                                              │
├─────┼────────────────────────────────────────────────────────────┼──────────────────────────────────────────────┤
│ 3   │ id: chinese-cookbook                                       │ 1. cvalues                                   │
│     │                                                            │ 2. cbbq-lite                                 │
│     │ Benchmarks for Chinese Language                            │                                              │
│     │ To test model's ability in answering safety and trust      │                                              │
│     │ questions using Chinese languages                          │                                              │
├─────┼────────────────────────────────────────────────────────────┼──────────────────────────────────────────────┤
│ 4   │ id: truthful-cookbook                                      │ 1. truthfulqa-mcq                            │
│     │                                                            │ 2. truthfulqa-open-ended                     │
│     │ Truthful Cookbook                                          │                                              │
│     │ This Cookbook will evaluated the truthfulness of my Model  │                                              │
├─────┼────────────────────────────────────────────────────────────┼──────────────────────────────────────────────┤
│ 5   │ id: evaluation-catalogue-cookbook                          │ 1. truthfulqa-mcq                            │
│     │                                                            │ 2. bbq                                       │
│     │ IMDA's LLM Evaluation Catalogue                            │ 3. real-toxicity-prompts-completion          │
│     │ This is a test cookbook for evaluation catalogue.          │ 4. fairness-uciadult                         │
│     │                                                            │ 5. enron-email                               │
│     │                                                            │ 6. advglue-mnli                              │
│     │                                                 

# Run Recipe(s)

We can run multiple recipes on multiple endpoints using `api_create_recipe_executor` as shown below.
- We can use recipe id to identify the recipe in this function.
- The results will be stored in `moonshot/data/results`

In [12]:
recipes = ["item-category", "bbq"]
endpoints = ["test-openai-endpoint"]
num_of_prompts = 5 # use a smaller number to test out the function

bm_executor = api_create_recipe_executor(
    "my new recipe executor",
    recipes,
    endpoints,
    num_of_prompts
)

await bm_executor.execute()
show_recipe_results(recipes, endpoints, bm_executor.results, bm_executor.results_file, bm_executor.duration)

bm_executor.close_executor()

Established connection to database (../moonshot/data/databases/recipe-my-new-recipe-executor.db)
🔃 Running recipes (my new recipe executor)... do not close this terminal.
You can start a new terminal to continue working.
Running recipe item-category... (1/2)
Part 0: Loading asyncio running loop...
Part 1: Loading various recipe instances...
Load recipe instance took 0.0001s


DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
DEBUG:openai:api_version=None data='{"model": "gpt-3.5-turbo", "messages": [{"role": "user", "content": "Answer this question:\\nWhat is an apple?\\nA:"}], "temperature": 0}' message='Post details'


Load recipe endpoints instances took 1.1038s
Load metrics took 0.0142s
Part 2: Building and executing generator pipeline for predicting prompts...
Predicting prompt 1 [test-openai-endpoint]


INFO:openai:message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=1063 request_id=req_907ae237292e96784e1acd027cb562a5 response_code=200
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
DEBUG:openai:api_version=None data='{"model": "gpt-3.5-turbo", "messages": [{"role": "user", "content": "Answer this question:\\nWhat is a chair?\\nA:"}], "temperature": 0}' message='Post details'


[Prompt 1] took 1.5534s
Predicting prompt 2 [test-openai-endpoint]


...

[Prompt 5] took 0.4643s
Predicting prompts for recipe [bbq] took 6.0781s
Part 3: Sort the recipe predictions into groups
Sort the recipe predictions into groups for recipe [bbq] took 0.0000s
Part 4: Performing metrics calculation
Running metrics for conn_id (test-openai-endpoint), recipe_id (bbq), dataset_id (bbq-ambiguous), prompt_template_id (bbq-template)
Running [get_results] took 0.0000s
Running metrics for conn_id (test-openai-endpoint), recipe_id (bbq), dataset_id (bbq-disamb), prompt_template_id (bbq-template)
Running [get_results] took 0.0000s
Performing metrics calculation for recipe [bbq] took 0.0000s


┏━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃   ┃ Recipe        ┃ test-openai-endpoint                                                                        ┃
┡━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 1 │ item-category │ {('test-dataset', 'test-prompt-template'): [{'exact_str_match': 0.0}, {'rouge': {'rouge-1': │
│   │               │ {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': │
│   │               │ 0.0, 'p': 0.0, 'f': 0.0}}}]}                                                                │
├───┼───────────────┼─────────────────────────────────────────────────────────────────────────────────────────────┤
│ 2 │ bbq           │ {('bbq-ambiguous', 'bbq-template'): [{'exact_str_match': 0.6}], ('bbq-disamb',              │
│   │               │ 'bbq-template'): [{'exact_str_match': 1.0}]}                                                │
└───┴───────────────┴─────────────────────────────────────────────────────────────────────────────────────────────┘

Results saved in ../moonshot/data/results/recipe-my-new-recipe-executor.json

==================================================
Time taken to run: 68s
==================================================

Closed connection to database (../moonshot/data/databases/recipe-my-new-recipe-executor.db)


# Run a cookbook

To run a cookbook, we can use `api_create_cookbook_executor`. 
- We can run multiple cookbooks on multiple endpoints.
- We can use cookbook id to identify the cookbook in this function.
- The results will be stored in `moonshot/data/results/`

In [13]:
cookbooks = ["test-category-cookbook"]
endpoints = ["test-openai-endpoint"]
num_of_prompts = 1

bm_executor = api_create_cookbook_executor(
    "my new cookbook executor",
    cookbooks,
    endpoints,
    num_of_prompts
)

await bm_executor.execute()
show_cookbook_results(cookbooks, endpoints, bm_executor.results, bm_executor.results_file, bm_executor.duration)

bm_executor.close_executor()

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
DEBUG:openai:api_version=None data='{"model": "gpt-3.5-turbo", "messages": [{"role": "user", "content": "Answer this question:\\nWhat is an apple?\\nA:"}], "temperature": 0}' message='Post details'


Established connection to database (../moonshot/data/databases/cookbook-my-new-cookbook-executor.db)
🔃 Running cookbooks (my new cookbook executor)... do not close this terminal.
You can start a new terminal to continue working.
Running cookbook test-category-cookbook... (1/1)
Part 1: Loading various cookbook instances...
Load cookbook instance took 0.0002s
Part 2: Executing cookbook recipes...
Running recipe item-category... (1/1)
Part 0: Loading asyncio running loop...
Part 1: Loading various recipe instances...
Load recipe instance took 0.0001s
Load recipe endpoints instances took 0.0008s
Load metrics took 0.0007s
Part 2: Building and executing generator pipeline for predicting prompts...
Predicting prompt 1 [test-openai-endpoint]


INFO:openai:message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=2581 request_id=req_03696d73ee9e07a4c8776bb3dae82f11 response_code=200


[Prompt 1] took 3.1262s
Predicting prompts for recipe [item-category] took 3.1315s
Part 3: Sort the recipe predictions into groups
Sort the recipe predictions into groups for recipe [item-category] took 0.0000s
Part 4: Performing metrics calculation
Running metrics for conn_id (test-openai-endpoint), recipe_id (item-category), dataset_id (test-dataset), prompt_template_id (test-prompt-template)
Running [get_results] took 0.0000s
Running [get_results] took 0.0002s
Performing metrics calculation for recipe [item-category] took 0.0002s
Executing cookbook [test-category-cookbook] took 3.1359s


┏━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃   ┃ Cookbook               ┃ Recipe        ┃ test-openai-endpoint                                               ┃
┡━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 1 │ test-category-cookbook │ item-category │ {('test-dataset', 'test-prompt-template'): [{'exact_str_match':    │
│   │                        │               │ 0.0}, {'rouge': {'rouge-1': {'r': 0.0, 'p': 0.0, 'f': 0.0},        │
│   │                        │               │ 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.0,   │
│   │                        │               │ 'p': 0.0, 'f': 0.0}}}]}                                            │
└───┴────────────────────────┴───────────────┴────────────────────────────────────────────────────────────────────┘

Results saved in ../moonshot/data/results/cookbook-my-new-cookbook-executor.json

==================================================
Time taken to run: 71s
==================================================

Closed connection to database (../moonshot/data/databases/cookbook-my-new-cookbook-executor.db)


# List all runs

Every run will be stored in Moonshot. You can list down your historical run using `api_get_all_executor`.

Runs are very useful in some scenarios. For examples:

1. Your network got interrupted and your run is stopped half way.
2. You want to re-run a specific run as you updated your model at the same endpoint.

In [14]:
executors_list = api_get_all_executor()
list_runs(executors_list)

Established connection to database (../moonshot/data/databases/recipe-my-new-recipe-executor.db)
Closed connection to database (../moonshot/data/databases/recipe-my-new-recipe-executor.db)
Established connection to database (../moonshot/data/databases/cookbook-my-new-cookbook-executor.db)
Closed connection to database (../moonshot/data/databases/cookbook-my-new-cookbook-executor.db)


┏━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ No. ┃ Run id                                ┃ Contains                                                        ┃
┡━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 1   │ id: recipe-my-new-recipe-executor     │ Recipes:                                                        │
│     │                                       │ ['item-category', 'bbq']                                        │
│     │                                       │                                                                 │
│     │                                       │ Endpoints:                                                      │
│     │                                       │ ['test-openai-endpoint']                                        │
│     │                                       │                                                                 │
│     │                                       │ Number of Prompts:                                              │
│     │                                       │ 5                                                               │
│     │                                       │                                                                 │
│     │                                       │ Database path:                                                  │
│     │                                       │ ../moonshot/data/databases/recipe-my-new-recipe-executor.db     │
├─────┼───────────────────────────────────────┼─────────────────────────────────────────────────────────────────┤
│ 2   │ id: cookbook-my-new-cookbook-executor │ Cookbooks:                                                      │
│     │                                       │ ['test-category-cookbook']                                      │
│     │                                       │                                                                 │
│     │                                       │ Endpoints:                                                      │
│     │                                       │ ['test-openai-endpoint']                                        │
│     │                                       │                                                                 │
│     │                                       │ Number of Prompts:                                              │
│     │                                       │ 1                                                               │
│     │                                       │                                                                 │
│     │                                       │ Database path:                                                  │
│     │                                       │ ../moonshot/data/databases/cookbook-my-new-cookbook-executor.db │
└─────┴───────────────────────────────────────┴─────────────────────────────────────────────────────────────────┘

## Resume a run

To resume a run, you can use `api_load_executor`.

In [15]:
# Resume a recipe run
run_id = "recipe-my-new-recipe-executor" # replace this with one of the run IDs shown above
bm_executor = api_load_executor(run_id)
await bm_executor.execute()
show_recipe_results(bm_executor.recipes, bm_executor.endpoints, bm_executor.results, bm_executor.results_file, bm_executor.duration)
bm_executor.close_executor()

Established connection to database (../moonshot/data/databases/recipe-my-new-recipe-executor.db)
🔃 Running recipes (my new recipe executor)... do not close this terminal.
You can start a new terminal to continue working.
Running recipe item-category... (1/2)
Part 0: Loading asyncio running loop...
Part 1: Loading various recipe instances...
Load recipe instance took 0.0002s
Load recipe endpoints instances took 0.0008s
Load metrics took 0.0004s
Part 2: Building and executing generator pipeline for predicting prompts...
Predicting prompts for recipe [item-category] took 0.0011s
Part 3: Sort the recipe predictions into groups
Sort the recipe predictions into groups for recipe [item-category] took 0.0000s
Part 4: Performing metrics calculation
Running metrics for conn_id (test-openai-endpoint), recipe_id (item-category), dataset_id (test-dataset), prompt_template_id (test-prompt-template)
Running [get_results] took 0.0000s
Running [get_results] took 0.0004s
Performing metrics calculation f

┏━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃   ┃ Recipe        ┃ test-openai-endpoint                                                                        ┃
┡━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 1 │ item-category │ {('test-dataset', 'test-prompt-template'): [{'exact_str_match': 0.0}, {'rouge': {'rouge-1': │
│   │               │ {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': │
│   │               │ 0.0, 'p': 0.0, 'f': 0.0}}}]}                                                                │
├───┼───────────────┼─────────────────────────────────────────────────────────────────────────────────────────────┤
│ 2 │ bbq           │ {('bbq-ambiguous', 'bbq-template'): [{'exact_str_match': 0.6}], ('bbq-disamb',              │
│   │               │ 'bbq-template'): [{'exact_str_match': 1.0}]}                                                │
└───┴───────────────┴─────────────────────────────────────────────────────────────────────────────────────────────┘

Results saved in ../moonshot/data/results/recipe-my-new-recipe-executor.json

==================================================
Time taken to run: 71s
==================================================

Closed connection to database (../moonshot/data/databases/recipe-my-new-recipe-executor.db)


In [16]:
# Resume a cookbook run
run_id = "cookbook-my-new-cookbook-executor" # replace this with one of the run IDs shown above
bm_executor = api_load_executor(run_id)
await bm_executor.execute()
show_cookbook_results(bm_executor.recipes, bm_executor.endpoints, bm_executor.results, bm_executor.results_file, bm_executor.duration)
bm_executor.close_executor()

Established connection to database (../moonshot/data/databases/cookbook-my-new-cookbook-executor.db)
🔃 Running cookbooks (my new cookbook executor)... do not close this terminal.
You can start a new terminal to continue working.
Running cookbook test-category-cookbook... (1/1)
Part 1: Loading various cookbook instances...
Load cookbook instance took 0.0002s
Part 2: Executing cookbook recipes...
Running recipe item-category... (1/1)
Part 0: Loading asyncio running loop...
Part 1: Loading various recipe instances...
Load recipe instance took 0.0001s
Load recipe endpoints instances took 0.0004s
Load metrics took 0.0004s
Part 2: Building and executing generator pipeline for predicting prompts...
Predicting prompts for recipe [item-category] took 0.0009s
Part 3: Sort the recipe predictions into groups
Sort the recipe predictions into groups for recipe [item-category] took 0.0000s
Part 4: Performing metrics calculation
Running metrics for conn_id (test-openai-endpoint), recipe_id (item-categ

┏━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃   ┃ Cookbook               ┃ Recipe        ┃ test-openai-endpoint                                               ┃
┡━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 1 │ test-category-cookbook │ item-category │ {('test-dataset', 'test-prompt-template'): [{'exact_str_match':    │
│   │                        │               │ 0.0}, {'rouge': {'rouge-1': {'r': 0.0, 'p': 0.0, 'f': 0.0},        │
│   │                        │               │ 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.0,   │
│   │                        │               │ 'p': 0.0, 'f': 0.0}}}]}                                            │
└───┴────────────────────────┴───────────────┴────────────────────────────────────────────────────────────────────┘

Results saved in ../moonshot/data/results/cookbook-my-new-cookbook-executor.json

==================================================
Time taken to run: 71s
==================================================

Closed connection to database (../moonshot/data/databases/cookbook-my-new-cookbook-executor.db)


Red Teaming

Create a Red Teaming session

In [17]:
endpoints = ["test-openai-endpoint"]

my_rt_session = api_create_session(
    "My Red Teaming Session",
    "Creating a new red teaming description",
    endpoints,
)

session_id = my_rt_session.metadata.session_id
show_session(my_rt_session)

Established connection to database (../moonshot/data/sessions/my-red-teaming-session_20240319-220543.db)
Established connection to database (../moonshot/data/sessions/my-red-teaming-session_20240319-220543.db)


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Session Id                             ┃ Session Info                           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ my-red-teaming-session_20240319-220543 │ Name:                                  │
│                                        │ My Red Teaming Session                 │
│                                        │                                        │
│                                        │ Description:                           │
│                                        │ Creating a new red teaming description │
│                                        │                                        │
│                                        │ Endpoints:                             │
│                                        │ ['test-openai-endpoint']               │
│                                        │                                        │
│                                        │ Context Strategy:                      │
│                                        │ None                                   │
│                                        │                                        │
│                                        │ Prompt Template:                       │
│                                        │ None                                   │
│                                        │                                        │
│                                        │                                        │
└────────────────────────────────────────┴────────────────────────────────────────┘

Send prompt to the endpoints

In [18]:
prompt = "What is the largest fruit"

await api_send_prompt(session_id, prompt)

show_session_chats(api_get_session_chats_by_session_id(session_id))

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
DEBUG:openai:api_version=None data='{"model": "gpt-3.5-turbo", "messages": [{"role": "user", "content": "What is the largest fruit"}], "temperature": 0}' message='Post details'


Established connection to database (../moonshot/data/sessions/my-red-teaming-session_20240319-220543.db)
Predicting prompt 1 [test-openai-endpoint]


INFO:openai:message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=627 request_id=req_f31165685d5e79d2e95a1cf2d8f79517 response_code=200


[Prompt 1] took 1.0076s
Established connection to database (../moonshot/data/sessions/my-red-teaming-session_20240319-220543.db)


┏━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ No. ┃ Endpoint             ┃ Contains                                                                           ┃
┡━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 1   │ test-openai-endpoint │ Chat Record Id:                                                                    │
│     │                      │ 1                                                                                  │
│     │                      │                                                                                    │
│     │                      │ Connection Id:                                                                     │
│     │                      │ None                                                                               │
│     │                      │                                                                                    │
│     │                      │ Context Strategy:                                                                  │
│     │                      │ None                                                                               │
│     │                      │                                                                                    │
│     │                      │ Prompt Template:                                                                   │
│     │                      │ None                                                                               │
│     │                      │                                                                                    │
│     │                      │ Prompt                                                                             │
│     │                      │ What is the largest fruit                                                          │
│     │                      │                                                                                    │
│     │                      │ Prepared Prompt:                                                                   │
│     │                      │ What is the largest fruit                                                          │
│     │                      │                                                                                    │
│     │                      │ Predicted Result:                                                                  │
│     │                      │ The largest fruit in the world is the jackfruit, which can weigh up to 80 pounds   │
│     │                      │ and grow up to 3 feet in length.                                                   │
│     │                      │                                                                                    │
│     │                      │ Duration:                                                                          │
│     │                      │ 1.00755404200754s                                                                  │
│     │                      │                                                                                    │
│     │                      │ Prompt Time:                                                                       │
│     │                      │ 03/19/2024, 22:05:43                                                               │
│     │                      │                                                                                    │
│     │                      │                                                                                    │
└─────┴──────────────────────┴────────────────────────────────────────────────────────────────────────────────────┘

Set Context Strategy and Prompt Template.

In [19]:
context_strategy = "add_previous_prompt"
prompt_template = "test-prompt-template"

api_update_context_strategy(session_id, context_strategy)
api_update_prompt_template(session_id, prompt_template)

# Get updated session
updated_session = api_get_session(session_id)
show_session(updated_session)

Established connection to database (../moonshot/data/sessions/my-red-teaming-session_20240319-220543.db)
Established connection to database (../moonshot/data/sessions/my-red-teaming-session_20240319-220543.db)
Established connection to database (../moonshot/data/sessions/my-red-teaming-session_20240319-220543.db)
Established connection to database (../moonshot/data/sessions/my-red-teaming-session_20240319-220543.db)


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Session Id                             ┃ Session Info                           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ my-red-teaming-session_20240319-220543 │ Name:                                  │
│                                        │ My Red Teaming Session                 │
│                                        │                                        │
│                                        │ Description:                           │
│                                        │ Creating a new red teaming description │
│                                        │                                        │
│                                        │ Endpoints:                             │
│                                        │ ['test-openai-endpoint']               │
│                                        │                                        │
│                                        │ Context Strategy:                      │
│                                        │ add_previous_prompt                    │
│                                        │                                        │
│                                        │ Prompt Template:                       │
│                                        │ test-prompt-template                   │
│                                        │                                        │
│                                        │                                        │
└────────────────────────────────────────┴────────────────────────────────────────┘

In [20]:
prompt = "What is the largest animal"

await api_send_prompt(session_id, prompt)

show_session_chats(api_get_session_chats_by_session_id(session_id))

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
DEBUG:openai:api_version=None data='{"model": "gpt-3.5-turbo", "messages": [{"role": "user", "content": "Answer this question:\\nWhat is the largest animalWhat is the largest fruit\\n\\nA:"}], "temperature": 0}' message='Post details'


Established connection to database (../moonshot/data/sessions/my-red-teaming-session_20240319-220543.db)
Predicting prompt 1 [test-openai-endpoint]


INFO:openai:message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=400 request_id=req_df46966da9212fec0310a206a45953a8 response_code=200


[Prompt 1] took 0.7659s
Established connection to database (../moonshot/data/sessions/my-red-teaming-session_20240319-220543.db)


┏━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ No. ┃ Endpoint             ┃ Contains                                                                           ┃
┡━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 1   │ test-openai-endpoint │ Chat Record Id:                                                                    │
│     │                      │ 1                                                                                  │
│     │                      │                                                                                    │
│     │                      │ Connection Id:                                                                     │
│     │                      │ None                                                                               │
│     │                      │                                                                                    │
│     │                      │ Context Strategy:                                                                  │
│     │                      │ None                                                                               │
│     │                      │                                                                                    │
│     │                      │ Prompt Template:                                                                   │
│     │                      │ None                                                                               │
│     │                      │                                                                                    │
│     │                      │ Prompt                                                                             │
│     │                      │ What is the largest fruit                                                          │
│     │                      │                                                                                    │
│     │                      │ Prepared Prompt:                                                                   │
│     │                      │ What is the largest fruit                                                          │
│     │                      │                                                                                    │
│     │                      │ Predicted Result:                                                                  │
│     │                      │ The largest fruit in the world is the jackfruit, which can weigh up to 80 pounds   │
│     │                      │ and grow up to 3 feet in length.                                                   │
│     │                      │                                                                                    │
│     │                      │ Duration:                                                                          │
│     │                      │ 1.00755404200754s                                                                  │
│     │                      │                                                                                    │
│     │                      │ Prompt Time:                                                                       │
│     │                      │ 03/19/2024, 22:05:43                                                               │
│     │                      │                                                                                    │
│     │                      │                                                                                    │
├─────┼──────────────────────┼────────────────────────────────────────────────────────────────────────────────────┤
│ 1   │ test-openai-endpoint │ Chat Record Id:                                                                    │
│     │                      │ 2                        

List all Context Strategies

In [21]:
context_strategies = api_get_all_context_strategy_name()
list_context_strategy(context_strategies)

┏━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃ No. ┃ Context Strategies  ┃
┡━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ 1   │ add_previous_prompt │
└─────┴─────────────────────┘

List all Prompt Templates

In [22]:
prompt_templates = api_get_all_prompt_template_detail()
list_prompt_templates(prompt_templates)

┏━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ No. ┃ Prompt Templates                                                                                          ┃
┡━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 1   │ Name:                                                                                                     │
│     │ question-answer-template1                                                                                 │
│     │                                                                                                           │
│     │ Description:                                                                                              │
│     │ This is a common template used for Q&A benchmarks.                                                        │
│     │                                                                                                           │
│     │ Template:                                                                                                 │
│     │ Q: "{{ prompt }}"                                                                                         │
│     │ Answer (Use the options in the question):                                                                 │
│     │                                                                                                           │
│     │                                                                                                           │
├─────┼───────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ 2   │ Name:                                                                                                     │
│     │ tamil-templatenewsclassification                                                                          │
│     │                                                                                                           │
│     │ Description:                                                                                              │
│     │ This template is used for Tamil News Classification for the Tamil News dataset.                           │
│     │                                                                                                           │
│     │ Template:                                                                                                 │
│     │ "{{ prompt }}" மேற்கொண்ட செய்தி எந்த பகுப்பை சேர்ந்தது? 'Only answer with either விளையாட்டு', 'உலகம்', 'இந்தியா', 'தமிழ்நாடு',        │
│     │ 'அரசியல்' or 'சினிமா'.                                                                                         │
│     │                                                                                                           │
│     │                                                                                                           │
├─────┼───────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ 3   │ Name:                                                                                                     │
│     │ advglue-templateqqp                                                                                       │
│     │                                                                                                           │
│     │ Description:                                                                                              │
│     │ This template is used for the QQP dataset. The task is to determine whether a pair of questions are       │
│     │ semanitically equivalent.                                                                                 │
│     │                                                                                                           │
│     │ Template:      

List all session names

In [23]:
sessions = api_get_all_session_detail()
list_sessions(sessions)

Established connection to database (../moonshot/data/sessions/my-red-teaming-session_20240319-220543.db)


┏━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ No. ┃ Session id                                 ┃ Contains                                 ┃
┡━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 1   │ id: my-red-teaming-session_20240319-220543 │ Name:                                    │
│     │                                            │ My Red Teaming Session                   │
│     │                                            │                                          │
│     │                                            │ Description:                             │
│     │                                            │ Creating a new red teaming description   │
│     │                                            │                                          │
│     │                                            │ Created:                                 │
│     │                                            │ 20240319-220543                          │
│     │                                            │                                          │
│     │                                            │ Context Strategy:                        │
│     │                                            │ add_previous_prompt                      │
│     │                                            │                                          │
│     │                                            │ Prompt Template:                         │
│     │                                            │ test-prompt-template                     │
│     │                                            │                                          │
│     │                                            │ Chat IDs:                                │
│     │                                            │ ['test_openai_endpoint_20240319_220543'] │
│     │                                            │                                          │
│     │                                            │                                          │
└─────┴────────────────────────────────────────────┴──────────────────────────────────────────┘